In [2]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.6/766.6 MB 4.5 MB/s eta 0:00:0000:0100:08
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 14.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 11.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 4.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 13.9 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.0 MB/s eta 0:00:0000:0100:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cesty k dátam
file_paths = {
    'wave1': "1. vlna všetko 28-11-2024.xlsx",
    'wave2': "2. vlna všetko 28-11-2024.xlsx",
    'wave3': "3. vlna všetko 28-11-2024.xlsx",
    'wave4': "4. vlna všetko 28-11-2024.xlsx"
}

# Pomocná funkcia na označenie neplatných textov
def is_invalid_text(val):
    invalid_prefixes = ['TO:', 'OA:', 'INVALID_PREFIX']
    if pd.isna(val):
        return True
    if isinstance(val, str):
        for prefix in invalid_prefixes:
            if val.strip().startswith(prefix):
                return True
    return False

# Funkcia na spracovanie stĺpcov, kde môže text presahovať
def process_continuing_texts(df, col1, col2):
    df[col2] = df.apply(
        lambda row: np.nan if pd.isna(row[col2]) and isinstance(row[col1], str) and len(row[col1]) > 25 else row[col2],
        axis=1
    )
    return df

# Funkcia na doplnenie hodnot v stĺpci 'DRG výkony'
def impute_drg_values(df, threshold=0.05, k_neighbors=5):
    drg_column = 'DRG výkony'
    if drg_column not in df.columns:
        return df

    missing_ratio = df[drg_column].isnull().mean()

    if missing_ratio < threshold:
        most_common_value = df[drg_column].mode()[0]
        df[drg_column] = df[drg_column].fillna(most_common_value)
        print(f"Stĺpec '{drg_column}': doplnených hodnôt pomocou najčastejšej hodnoty (modus).")
    else:
        print(f"Stĺpec '{drg_column}': doplnenie pomocou KNN imputer.")
        imputer = KNNImputer(n_neighbors=k_neighbors)
        df[[drg_column]] = imputer.fit_transform(df[[drg_column]])
        print(f"Stĺpec '{drg_column}': doplnené hodnoty pomocou KNN.")
    return df

# Hlavná funkcia na doplnenie hodnôt
def impute_missing_values(df, threshold=0.05, k_neighbors=5, text_columns=None):
    df_filled = df.copy()

    # Odstránenie stĺpcov s viac ako 75 % chýbajúcich hodnôt
    missing_ratios = df.isnull().mean()
    to_drop = missing_ratios[missing_ratios > 0.75].index.tolist()
    df_filled.drop(columns=to_drop, inplace=True)
    print(f" Odstránené {len(to_drop)} stĺpcov s >75% chýbajúcich hodnôt: {to_drop}")

    numeric_cols = df_filled.select_dtypes(include=['number']).columns
    categorical_cols = df_filled.select_dtypes(exclude=['number']).columns

    #  Odstránenie medzier z textových stĺpcov
    for col in categorical_cols:
        df_filled[col] = df_filled[col].apply(lambda x: str(x).strip() if pd.notna(x) else x)

    # Spracovanie presahujúcich textov
    if text_columns:
        for col1, col2 in text_columns:
            df_filled = process_continuing_texts(df_filled, col1, col2)

    # Numerické stĺpce
    for col in numeric_cols:
        missing_ratio = df_filled[col].isnull().mean()
        missing_count = df_filled[col].isnull().sum()
        if missing_ratio > 0:
            if missing_ratio < threshold:
                df_filled[col] = df_filled[col].fillna(df_filled[col].mean())
                print(f" {col}: doplnených {missing_count} hodnôt pomocou priemeru")

    knn_numeric = [col for col in numeric_cols if df_filled[col].isnull().mean() >= threshold]
    if knn_numeric:
        knn_data = df_filled[knn_numeric].dropna(axis=1, how='all')
        if not knn_data.empty:
            imputer = KNNImputer(n_neighbors=k_neighbors)
            imputed = imputer.fit_transform(knn_data)
            df_filled[knn_data.columns] = imputed
            for col in knn_data.columns:
                print(f" {col}: doplnené hodnoty pomocou KNN")

    # Špeciálne spracovanie 'Návyková anamnéza'
    if 'Návyková anamnéza' in df_filled.columns:
        col = 'Návyková anamnéza'
        prefix = "NA:"

        suffix_values = df_filled[col].apply(
            lambda x: str(x).split(":", 1)[1].strip() if isinstance(x, str) and ":" in x else ""
        )

        invalid_mask = suffix_values == ""
        valid_mask = ~invalid_mask
        missing_ratio = invalid_mask.mean()
        missing_count = invalid_mask.sum()

        print(f" {col}: {missing_count} hodnôt je prázdnych (iba prefix '{prefix}')")

        if missing_ratio < threshold:
            most_common_suffix = suffix_values[valid_mask].mode()[0]
            filled_suffix = suffix_values.copy()
            filled_suffix[invalid_mask] = most_common_suffix
            df_filled[col] = prefix + " " + filled_suffix
            print(f" {col}: doplnené hodnoty pomocou najčastejšej hodnoty (modus)")
        else:
            print(f"{col}: doplňovanie pomocou TF-IDF + KNN")
            text_data = suffix_values.fillna("").apply(lambda x: x.strip())

            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform(text_data)
            embeddings = tfidf_matrix.toarray()
            embeddings = np.nan_to_num(embeddings)

            imputer = KNNImputer(n_neighbors=k_neighbors)
            imputed_embeddings = imputer.fit_transform(embeddings)

            notna_texts = suffix_values[valid_mask].unique()
            notna_vecs = vectorizer.transform(notna_texts).toarray()

            filled_values = []
            for i, val in enumerate(suffix_values):
                if invalid_mask.iloc[i]:
                    filled_vec = imputed_embeddings[i]
                    closest_idx = cosine_similarity([filled_vec], notna_vecs).argmax()
                    filled_suffix = notna_texts[closest_idx]
                    filled_values.append(f"{prefix} {filled_suffix}")
                else:
                    filled_values.append(f"{prefix} {val}")

            df_filled[col] = filled_values
            print(f" {col}: doplnené hodnoty pomocou TF-IDF + cosine similarity")

    # Ostatné kategóriové stĺpce
    for col in categorical_cols:
        if col == 'Návyková anamnéza':
            continue  # už spracované vyššie

        col_values = df_filled[col]
        invalid_mask = col_values.apply(is_invalid_text)
        valid_mask = ~invalid_mask
        missing_ratio = invalid_mask.mean()
        missing_count = invalid_mask.sum()

        print(f"Zistené neplatné hodnoty v stĺpci '{col}': {df_filled[col][invalid_mask].unique()}")

        if missing_ratio > 0:
            if missing_ratio < threshold:
                most_common = col_values[valid_mask].mode()[0]
                df_filled.loc[invalid_mask, col] = most_common
                print(f" {col}: doplnených {missing_count} hodnôt pomocou najčastejšej hodnoty (mod)")
            else:
                print(f" {col}: doplňovanie pomocou TF-IDF + KNN")
                text_data = df_filled[col].fillna("")
                vectorizer = TfidfVectorizer()
                tfidf_matrix = vectorizer.fit_transform(text_data)
                embeddings = tfidf_matrix.toarray()
                embeddings = np.nan_to_num(embeddings)

                imputer = KNNImputer(n_neighbors=k_neighbors)
                imputed_embeddings = imputer.fit_transform(embeddings)

                notna_texts = df_filled.loc[valid_mask, col].unique()
                notna_vecs = vectorizer.transform(notna_texts).toarray()

                filled_values = []
                for i, orig in enumerate(df_filled[col]):
                    if is_invalid_text(orig):
                        filled_vec = imputed_embeddings[i]
                        closest_idx = cosine_similarity([filled_vec], notna_vecs).argmax()
                        filled_values.append(notna_texts[closest_idx])
                    else:
                        filled_values.append(orig)

                df_filled[col] = filled_values
                print(f" {col}: doplnené hodnoty pomocou TF-IDF + cosine similarity")

    #  Doplnenie stĺpca 'DRG výkony'
    df_filled = impute_drg_values(df_filled, threshold=threshold, k_neighbors=k_neighbors)

    return df_filled

# Spustenie na všetkých súboroch
data_dict = {}
for wave, path in file_paths.items():
    print(f"\n Spracovávam {wave}...")
    df_loaded = pd.read_excel(path)
    df_imputed = impute_missing_values(df_loaded)
    data_dict[wave] = df_imputed

    output_filename = f"{wave}_imputed.xlsx"
    df_imputed.to_excel(output_filename, index=False)
    print(f" Uložené: {output_filename}")


🌊 Spracovávam wave1...
❌ Odstránené 18 stĺpcov s >75% chýbajúcich hodnôt: ['Unnamed: 23', 'S-Chol first', 'S-Chol last', 'S-Chol min', 'S-Chol max', 'S-IgG first', 'S-IgG last', 'S-IgG min', 'S-IgG max', 'S-IgA first', 'S-IgA last', 'S-IgA min', 'S-IgA max', 'S-Ig M first', 'S-Ig M last', 'S-Ig M min', 'S-Ig M max', 'Typ vakcíny']
🔢 S-Bil-T first: doplnených 18 hodnôt pomocou priemeru
🔢 S-Bil-T last: doplnených 18 hodnôt pomocou priemeru
🔢 S-Bil-T min: doplnených 18 hodnôt pomocou priemeru
🔢 S-Bil-T max: doplnených 18 hodnôt pomocou priemeru
🔢 S-AST first: doplnených 9 hodnôt pomocou priemeru
🔢 S-AST last: doplnených 9 hodnôt pomocou priemeru
🔢 S-AST min: doplnených 9 hodnôt pomocou priemeru
🔢 S-AST max: doplnených 9 hodnôt pomocou priemeru
🔢 S-ALT first: doplnených 9 hodnôt pomocou priemeru
🔢 S-ALT last: doplnených 9 hodnôt pomocou priemeru
🔢 S-ALT min: doplnených 9 hodnôt pomocou priemeru
🔢 S-ALT max: doplnených 9 hodnôt pomocou priemeru
🔢 S-GMT first: doplnených 10 hodnôt pomocou p